In [1]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Define the date range and categories
dates = pd.date_range(start="2024-01-01", end="2024-07-31", freq='M')
categories = ["Groceries", "Housing", "Health", "Entertainment", "Insurance", "Transportation"]

# Generate dummy spending data
np.random.seed(42)
data = {
    "Date": np.tile(dates, len(categories)),
    "Category": np.repeat(categories, len(dates)),
    "Spending": np.random.uniform(100, 1000, len(dates) * len(categories))
}
df = pd.DataFrame(data)

# Generate dummy CPI data with values between 1.01 and 1.12
cpi_data = {
    "Date": dates,
    "CPI": np.random.uniform(1.01, 1.12, len(dates))
}
cpi_df = pd.DataFrame(cpi_data)

# Merge the dataframes
merged_df = pd.merge(df, cpi_df, on="Date")
merged_df["Adjusted Spending"] = merged_df["Spending"] * merged_df["CPI"]

# Prepare data for machine learning model (e.g., linear regression) using 6 previous months
def prepare_ml_data(merged_df, category):
    category_data = merged_df[merged_df['Category'] == category]
    category_data = category_data[['Date', 'Spending']].set_index('Date').resample('M').sum()

    # Using the last 6 months to predict the next month
    for i in range(1, 7):
        category_data[f'Lag{i}'] = category_data['Spending'].shift(i)

    category_data = category_data.dropna()  # Drop any rows with missing values

    X = category_data[[f'Lag{i}' for i in range(1, 7)]]
    y = category_data['Spending']
    
    return X, y

# Predict the next month's CPI using the previous 6 months
def predict_next_cpi(cpi_df):
    cpi_data = cpi_df.set_index('Date').resample('M').mean()
    for i in range(1, 7):
        cpi_data[f'Lag{i}'] = cpi_data['CPI'].shift(i)

    cpi_data = cpi_data.dropna()

    X = cpi_data[[f'Lag{i}' for i in range(1, 7)]]
    y = cpi_data['CPI']

    model = LinearRegression()
    model.fit(X, y)

    last_6_cpi = X.iloc[-1].values.reshape(1, -1)
    predicted_cpi = model.predict(last_6_cpi)[0]

    next_month = X.index[-1] + pd.DateOffset(months=1)

    return next_month, predicted_cpi

# Predict expenses and adjusted spending for each category
def predict_expenses_and_adjusted_spending(merged_df, cpi_df, categories):
    predictions = []
    next_month, predicted_cpi = predict_next_cpi(cpi_df)

    for category in categories:
        X, y = prepare_ml_data(merged_df, category)
        
        model = LinearRegression()
        model.fit(X, y)

        # Predict the next month's spending
        last_6_months = X.iloc[-1].values.reshape(1, -1)
        predicted_spending = model.predict(last_6_months)[0]

        adjusted_spending = predicted_spending * predicted_cpi
        predictions.append({
            "Date": next_month, 
            "Category": category, 
            "Spending": predicted_spending, 
            "CPI": predicted_cpi, 
            "Adjusted Spending": adjusted_spending
        })

    return pd.DataFrame(predictions)

# Get the predictions
pred_df = predict_expenses_and_adjusted_spending(merged_df, cpi_df, categories)

# Add the predicted data to the merged dataframe
merged_df = pd.concat([merged_df, pred_df], ignore_index=True)

# Ensure that the dates are sorted correctly
merged_df = merged_df.sort_values(by="Date")

# Update the date range to include the predicted month
dates = pd.date_range(start="2024-01-01", end=pred_df['Date'].max(), freq='M')

# Display the dataframes
print("Dummy Spending Data:")
display(df)

print("\nCPI Data:")
display(cpi_df)

print("\nMerged Data with Adjusted Spending:")
display(merged_df)

print("\nPredicted Spending for Next Month:")
display(pred_df)

# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'By Category', 'value': 'Category'},
            {'label': 'By Month', 'value': 'Date'}
        ],
        value='Category',
        style={'width': '50%'}
    ),
    dcc.Dropdown(
        id='filter-value',
        style={'width': '50%', 'margin-top': '10px'}
    ),
    dcc.Graph(id='spending-chart')
])

# Callback to update the filter value options based on the filter type
@app.callback(
    Output('filter-value', 'options'),
    [Input('filter-type', 'value')]
)
def set_filter_value_options(filter_type):
    if filter_type == 'Category':
        return [{'label': cat, 'value': cat} for cat in categories]
    else:
        return [{'label': date.strftime("%B %Y"), 'value': date} for date in dates]

# Callback to update the graph based on the selected filter
@app.callback(
    Output('spending-chart', 'figure'),
    [Input('filter-type', 'value'), Input('filter-value', 'value')]
)
def update_graph(filter_type, filter_value):
    if filter_value is None:
        return {}

    if filter_type == 'Category':
        # Filter by category
        filtered_df = merged_df[merged_df['Category'] == filter_value]
        title = f'Spending and Adjusted Spending for {filter_value}'

        fig = go.Figure()

        if not filtered_df.empty:
            # Add actual spending data
            fig.add_trace(go.Bar(
                x=filtered_df['Date'].dt.strftime("%B %Y"),
                y=filtered_df['Spending'],
                name='Spending',
                marker_color='blue'
            ))

            # Add adjusted spending data
            fig.add_trace(go.Bar(
                x=filtered_df['Date'].dt.strftime("%B %Y"),
                y=filtered_df['Adjusted Spending'],
                name='Adjusted Spending',
                marker_color='green'
            ))

        fig.update_layout(barmode='group', title=title)
        
    else:
        # Filter by date (month)
        filtered_df = merged_df[merged_df['Date'] == pd.to_datetime(filter_value)]

        title = f'Spending and Adjusted Spending for {pd.to_datetime(filter_value).strftime("%B %Y")}'

        fig = go.Figure()

        if not filtered_df.empty:
            # Add actual spending data by category
            fig.add_trace(go.Bar(
                x=filtered_df['Category'],
                y=filtered_df['Spending'],
                name='Spending',
                marker_color='blue'
            ))

            # Add adjusted spending data by category
            fig.add_trace(go.Bar(
                x=filtered_df['Category'],
                y=filtered_df['Adjusted Spending'],
                name='Adjusted Spending',
                marker_color='green'
            ))

        fig.update_layout(barmode='group', title=title)

    return fig

# Run the app on port 8055
if __name__ == '__main__':
    app.run_server(debug=True, port=8055)


Dummy Spending Data:


C:\Users\mpena\anaconda3\envs\capstone\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\mpena\anaconda3\envs\capstone\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\mpena\anaconda3\envs\capstone\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\mpena\anaconda3\envs\capstone\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\mpena\anaconda3\envs\capstone\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\mpena\anaco

,Date,Category,Spending
0,2024-01-31,Groceries,437.086107
1,2024-02-29,Groceries,955.642876
2,2024-03-31,Groceries,758.794548
3,2024-04-30,Groceries,638.792636
4,2024-05-31,Groceries,240.416776
5,2024-06-30,Groceries,240.395068
6,2024-07-31,Groceries,152.275251
7,2024-01-31,Housing,879.558531
8,2024-02-29,Housing,641.003511
9,2024-03-31,Housing,737.265320



CPI Data:


,Date,CPI
0,2024-01-31,1.013783
1,2024-02-29,1.110025
2,2024-03-31,1.038466
3,2024-04-30,1.082877
4,2024-05-31,1.044288
5,2024-06-30,1.067207
6,2024-07-31,1.070138



Merged Data with Adjusted Spending:


,Date,Category,Spending,CPI,Adjusted Spending
0,2024-01-31,Groceries,437.086107,1.013783,443.110350
1,2024-01-31,Housing,879.558531,1.013783,891.681255
2,2024-01-31,Health,263.642470,1.013783,267.276185
3,2024-01-31,Entertainment,225.544475,1.013783,228.653095
4,2024-01-31,Insurance,633.173112,1.013783,641.899971
5,2024-01-31,Transportation,827.557613,1.013783,838.963623
11,2024-02-29,Transportation,374.152392,1.110025,415.318601
10,2024-02-29,Insurance,141.805371,1.110025,157.407542
9,2024-02-29,Entertainment,362.930184,1.110025,402.861666
7,2024-02-29,Housing,641.003511,1.110025,711.530078



Predicted Spending for Next Month:


,Date,Category,Spending,CPI,Adjusted Spending
0,2024-08-31,Groceries,152.275251,1.070138,162.955552
1,2024-08-31,Housing,291.105200,1.070138,311.522774
2,2024-08-31,Health,650.667605,1.070138,696.304215
3,2024-08-31,Entertainment,562.810995,1.070138,602.285506
4,2024-08-31,Insurance,969.068830,1.070138,1037.037506
5,2024-08-31,Transportation,545.659219,1.070138,583.930737
